In [1]:
import moviepy.editor as mp
import tempfile
import os
import pandas as pd
import speech_recognition as sr

In [2]:
# def transcribe_audio(audio_file_path, start_time, end_time):
#     # Initialize the recognizer
#     recognizer = sr.Recognizer()

#     # Load the audio file
#     with sr.AudioFile(audio_file_path) as audio_file:
#         # Use Google Web Speech API to recognize the audio
#         try:
#             audio_data = recognizer.record(audio_file)
#             transcription = recognizer.recognize_google(audio_data, language='nl-NL')
#             print("Start Time: {}, End Time: {}, Transcription: {}".format(start_time, end_time, transcription))
#             return transcription
#         except sr.UnknownValueError:
#             print("Google Speech Recognition could not understand the audio from {} to {}".format(start_time, end_time))
#             return None
#         except sr.RequestError as e:
#             print("Could not request results from Google Speech Recognition service from {} to {}; {}".format(start_time, end_time, e))
#             return None

In [3]:
video = r"C:\Users\mened\OneDrive\Desktop\ER22_AFL09_MXF.mov"
video_clip = mp.VideoFileClip(video)

In [4]:
data = pd.read_csv('Robinson22_structure.csv')

In [5]:
data = data[data['Episode name'] == 9]

In [6]:
data = data.drop_duplicates(subset=['Start Time (seconds)'], keep='first')

In [7]:
data = data.reset_index(drop=True)

In [8]:
data

,Instance name,Episode name,Act,Chapter,Segment,Segment title,Segment description,Start Time (seconds),End Time (seconds),SKO Start time,...,Madl (%) V 6+ jr Currency,Madl (%) V 6+ jr Uitgesteld,Madl (%) V 6+ jr Uitzenddag,Locations,Props,Actors,Emotions,Game Types,Actions,News Types
0,Survivor,9,Samensmelting,Idents,Vorige keer,NaN,NaN,0,4285,2022-10-24 18:38:00 +00:00,...,21.007156,28.165737,18.573592,"Locatie immuniteitsproef, Locatie captain's ba...","Plunder, Team paars (bandana), Team oranje (ba...","Nicolette Kluijver, Rick Brandsteder, Dennis W...",Joy,NaN,"Strategy, Argument, Commentary",NaN
1,Survivor,9,Samensmelting,Reality,Afscheid buiten de eilandraad,NaN,NaN,104,4285,2022-10-24 19:27:00 +00:00,...,24.530806,34.775482,22.485979,Kamp Noord,Team paars (bandana),"Iliass Ojja, Rick Brandsteder (VO)",Fear,NaN,"Team drop-out, Commentary",NaN
2,Survivor,9,Samensmelting,Reality,Small talk,NaN,NaN,192,4285,2022-10-24 19:47:00 +00:00,...,24.392356,32.658672,21.918551,Kamp Noord,Team paars (bandana),"Ferry Weertman, April Darby, Niels Oosthoek","Surprise, Caring, Disappointment",NaN,Diary,NaN
3,Survivor,9,Samensmelting,Reality,Strategie en bondjes,NaN,NaN,335,4285,2022-10-24 19:26:00 +00:00,...,23.093829,32.783058,21.138481,Kamp West,"Eten, Kookgerei, Team oranje (bandana)","Dennis Wilt, Harry Piekema, Vincent Vianen, Ch...",Amusement,NaN,Speculation,NaN
4,Survivor,9,Samensmelting,Reality,Persoonlijke verhalen,NaN,NaN,441,4285,2022-10-24 20:00:00 +00:00,...,28.155612,38.215986,25.229996,Kamp Zuid,"Eten, Team groen (bandana)","Niek Roozen, Dzifa Kusenuh, Thijs Boermans, Mi...","Sadness, Caring, Optimism",NaN,"Diary, Confession",NaN
5,Survivor,9,Samensmelting,Reality,Strategie en bondjes,NaN,NaN,629,4285,2022-10-24 20:06:00 +00:00,...,28.023524,39.117158,24.853655,NaN,"Perkament brief, Flessenpost",London Loy,NaN,NaN,NaN,NaN
6,Survivor,9,Samensmelting,Idents,Leader,NaN,NaN,640,4285,2022-10-24 20:04:00 +00:00,...,27.612184,38.427981,24.357481,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Survivor,9,Samensmelting,Reality,Strategie en bondjes,NaN,NaN,704,4285,2022-10-24 19:53:00 +00:00,...,28.045042,39.509907,23.858140,NaN,"Team paars (bandana), Team oranje (bandana), T...","London Loy, Ferry Weertman, April Darby, Dzifa...","Optimism, Anticipation",NaN,"Speculation, Diary, Commentary",NaN
8,Survivor,9,Samensmelting,Reality,Small talk,NaN,NaN,842,4285,2022-10-24 19:29:00 +00:00,...,25.820439,37.809531,24.220145,Locatie captain's battle,NaN,"Nicolette Kluijver, London Loy, Ferry Weertman...","Surprise, Excitement",NaN,Diary,NaN
9,Survivor,9,Samensmelting,Idents,Straks,NaN,NaN,949,4285,2022-10-24 19:01:00 +00:00,...,21.774097,28.465571,19.199559,Locatie captain's battle,"Eten, Koker","Nicolette Kluijver, London Loy",NaN,NaN,"Supsense, Diary",NaN


In [9]:
# for i, row in data.iterrows():
#     start_time = row['Start Time (seconds)']
    
#     if i < len(data) - 1:
#         end_time = data.loc[i + 1, 'Start Time (seconds)']
#     else:
#         end_time = start_time + video_clip.duration

#     print(start_time, end_time)

#     # Extract the audio segment based on start and end times
#     audio_segment = video_clip.audio.subclip(start_time, end_time)

#     # Write the audio segment to a temporary file
#     temp_audio_path = f"temp_audio_{start_time}_{end_time}.wav"
#     audio_segment.write_audiofile(temp_audio_path, codec='pcm_s16le')

#     # Transcribe the audio segment
#     transcription = transcribe_audio(temp_audio_path, start_time, end_time)

#     # Remove the temporary audio file
#     os.remove(temp_audio_path)

# # Close the video clip
# video_clip.close()

In [10]:
def process_episode(episode_path, data):
    def transcribe_audio(audio_file_path, start_time, end_time):

        recognizer = sr.Recognizer()

        with sr.AudioFile(audio_file_path) as audio_file:
            try:
                audio_data = recognizer.record(audio_file)
                transcription = recognizer.recognize_google(audio_data, language='nl-NL')
                return transcription
            except sr.UnknownValueError:
                return "Google Speech Recognition could not understand the audio."
            except sr.RequestError as e:
                return "Could not request results from Google Speech Recognition service: {}".format(e)

    video_clip = mp.VideoFileClip(episode_path)

    episode_name = os.path.splitext(os.path.basename(episode_path))[0]
    episode_folder = os.path.join(os.path.dirname(episode_path), episode_name)
    os.makedirs(episode_folder, exist_ok=True)

    for i, row in data.iterrows():
        start_time = row['Start Time (seconds)']
        
        if i < len(data) - 1:
            end_time = data.loc[i + 1, 'Start Time (seconds)']
        else:
            end_time = video_clip.duration

        audio_segment = video_clip.audio.subclip(start_time, end_time)

        fragment_name = f"fragment_{i + 1}"
        temp_audio_path = os.path.join(episode_folder, f"temp_audio_{fragment_name}.wav")
        audio_segment.write_audiofile(temp_audio_path, codec='pcm_s16le')

        transcription = transcribe_audio(temp_audio_path, start_time, end_time)
        transcription_file = os.path.join(episode_folder, f"transcription_{fragment_name}.txt")
        with open(transcription_file, 'w') as f:
            f.write(transcription)

        os.remove(temp_audio_path)

    video_clip.close()


In [11]:
process_episode(video, data)

MoviePy - Writing audio in C:\Users\mened\OneDrive\Desktop\ER22_AFL09_MXF\temp_audio_fragment_1.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\mened\OneDrive\Desktop\ER22_AFL09_MXF\temp_audio_fragment_2.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\mened\OneDrive\Desktop\ER22_AFL09_MXF\temp_audio_fragment_3.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\mened\OneDrive\Desktop\ER22_AFL09_MXF\temp_audio_fragment_4.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\mened\OneDrive\Desktop\ER22_AFL09_MXF\temp_audio_fragment_5.wav


...

MoviePy - Done.
MoviePy - Writing audio in C:\Users\mened\OneDrive\Desktop\ER22_AFL09_MXF\temp_audio_fragment_34.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\mened\OneDrive\Desktop\ER22_AFL09_MXF\temp_audio_fragment_35.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\mened\OneDrive\Desktop\ER22_AFL09_MXF\temp_audio_fragment_36.wav


MoviePy - Done.
MoviePy - Writing audio in C:\Users\mened\OneDrive\Desktop\ER22_AFL09_MXF\temp_audio_fragment_37.wav


MoviePy - Done.
